# Capítulo 6 - Diferentes tipos de dados - Parte 1

Esta parte foca em construir expressões, conhecer e revisar os tipos de dados ja conhecidos e novos do Spark. Abaixo seguem algumas dicas de materiais complementares para encontrar funções para transformação de dados:

https://spark.apache.org/docs/2.3.0/api/java/index.html?org/apache/spark/sql/Dataset.html

https://spark.apache.org/docs/2.4.0/sql-programming-guide.html

https://spark.apache.org/docs/latest/api/sql/index.html

### Sumário
 1. __Boleanos__
 2. __Números__
 3. __Strings__
 4. __Datas e timestamps__

__Obs.__: Por convenção, DF..., df..., df_... , são variáveis que armazenam DataFrames.

In [1]:
# Conjunto de dados usados para os exemplos que seguem
path = "file:///root/Spark_Certificacao/data/retail-data/by-day/2010-12-01.csv"
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load(path)
df.printSchema()
df.createOrReplaceTempView("dfTable")
df.show(4)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365| 

## Convertendo dados para os tipos do Spark

Uma tarefa que realizaremos é converter tipos nativos em tipos Spark. Fazemos isso usando a primeira função que apresentamos aqui, a função __lit__. Esta função converte um tipo de dado de outra linguagem para sua equivalente no Spark. Veja como podemos converter alguns tipos diferentes de valores do Python para seus respectivos tipos de Spark:

In [2]:
from pyspark.sql.functions import lit

# Tipos de dados do Python
print type(5)
print type("five")
print type(5.0)

# Usando a função lit para converter em Spark
df.select(lit(5), lit("five"), lit(5.0))

<type 'int'>
<type 'str'>
<type 'float'>


DataFrame[5: int, five: string, 5.0: double]

## Boleanos

Booleanos são essenciais quando se trata de análise de dados, porque eles são a base para toda a filtragem. As declarações booleanas consistem em quatro elementos: ___and___, ___or___, ___true___ e ___false___. Usamos essas estruturas simples para criar instruções lógicas que são avaliadas como verdadeiras ou falsas. Essas instruções são frequentemente usadas como requisitos condicionais para quando uma linha de dados deve passar no teste (avaliar para verdadeiro) ou então será filtrada.

Operadores lógicos em Spark:

In [ ]:
&  # and
|  # or
~  # not

In [3]:
from pyspark.sql.functions import col
df.where(col("InvoiceNo") != 536365)\
.select("InvoiceNo", "Description")\
.show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



Outra opção é especificar o predicado como uma expressão em uma string. Isso é válido para Python ou Scala. Note que isso também dá acesso a outra maneira de expressar "não é igual":

In [4]:
df.where("InvoiceNo <> 536365").show(5, False)

+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                  |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|536366   |22633    |HAND WARMER UNION JACK       |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536366   |22632    |HAND WARMER RED POLKA DOT    |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536367   |84879    |ASSORTED COLOUR BIRD ORNAMENT|32      |2010-12-01 08:34:00|1.69     |13047.0   |United Kingdom|
|536367   |22745    |POPPY'S PLAYHOUSE BEDROOM    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
|536367   |22748    |POPPY'S PLAYHOUSE KITCHEN    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
+---------+---------+-----------------------------+--------+----

Você pode especificar expressões booleanas com várias partes quando você usa ___and___ ou ___not___. No Spark, você deve sempre encadear o ___and___ e filtrar como um filtro sequencial. A razão para isso é que mesmo que as declarações booleanas sejam expressas em série (uma após a outra), o Spark achatará todos esses filtros em uma instrução e executará o filtro ao mesmo tempo, criando a declaração ___and___ para nós. Embora você possa especificar suas declarações explicitamente usando ___and___, se desejar, elas serão mais fáceis de entender e ler se forem especificadas em série. A instrução ___or___ precisa ser especificada na mesma declaração:

In [5]:
from pyspark.sql.functions import instr

priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()

# Equivalente em SQL:
# SELECT * FROM dfTable WHERE StockCode in ("DOT") AND(UnitPrice > 600 OR instr(Description, "POSTAGE") >= 1)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



\*A função __instr__ retorna o índice da primeira ocorrência de uma substring dada uma string.

Expressões booleanas não são reservadas apenas para filtros. Para filtrar um DataFrame, você também pode especificar apenas uma coluna booleana:

In [6]:
from pyspark.sql.functions import instr

DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1

df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter))\
                .where("isExpensive")\
                .select("unitPrice", "isExpensive").show(5)

# SELECT UnitPrice, (StockCode = 'DOT' AND
#        (UnitPrice > 600 OR instr(Description, "POSTAGE") >= 1)) as isExpensive
#        FROM dfTable
#        WHERE (StockCode = 'DOT' AND
#        (UnitPrice > 600 OR instr(Description, "POSTAGE") >= 1))

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



Observe como não precisávamos especificar nosso filtro como uma expressão e como poderíamos usar um nome de coluna sem nenhum trabalho extra. Se você estiver acostumado com SQL, todas essas instruções devem parecer bastante familiares. De fato, todos eles podem ser expressos como uma cláusula __where__. Na verdade, muitas vezes é mais fácil apenas expressar os filtros como instruções SQL do que usar a interface programática do DataFrame e o Spark SQL nos permite fazer isso sem pagar qualquer penalidade de desempenho.

In [7]:
from pyspark.sql.functions import expr

df.withColumn("isExpensive", expr("NOT UnitPrice <= 250"))\
                .where("isExpensive")\
                .select("Description", "UnitPrice").show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



Uma pegadinha que pode surgir é quando você estiver trabalhando com dados nulos ao criar expressões booleanas. Se houver um nulo em seus dados, você precisará tratar as coisas de maneira um pouco diferente. Veja como garantir a realização de um teste de equivalência com segurança nula:

In [8]:
df.where(col("Description").eqNullSafe("hello")).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



## Números

A segunda tarefa mais comum que você fará depois de filtrar as coisas é contar as coisas. Na maioria das vezes, precisamos apenas expressar nossa computação, e isso deve ser válido supondo que estamos trabalhando com tipos de dados numéricos. Vamos imaginar que, descobrimos que não registramos a quantidade de dados de varejo em nosso conjunto de dados e, a quantidade real é igual a:<br>
    <b>(a quantidade atual * o preço unitário)^2 + 5.</b> 
<br>
Isso também introduzirá nossa primeira função numérica como a função potência que eleva uma coluna a potência desejada:

In [9]:
from pyspark.sql.functions import expr, pow

fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



Repare que fomos capazes de multiplicar nossas colunas porque ambas eram numéricas. Naturalmente, podemos adicionar e subtrair, se necessário, também. Na verdade, podemos fazer tudo isso como um SQL.

In [10]:
df.selectExpr(
"CustomerId",
"(POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity").show(2)

# Equivalente em SQL:
# SELECT customerId, (POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity
# FROM dfTable

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [11]:
# Arredondando Valores
from pyspark.sql.functions import lit, round, bround

df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)

# Equivalente em SQL:
# SELECT round(2.5), bround(2.5)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



In [12]:
# Funções estatísticas podem ser aplicadas também. 
# Correlação entre duas colunas:
from pyspark.sql.functions import corr
df.stat.corr("Quantity", "UnitPrice")
df.select(corr("Quantity", "UnitPrice")).show()

# Equivalente em SQL:
# SELECT corr(Quantity, UnitPrice) FROM dfTable

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



Outra tarefa comum é calcular estatísticas-resumo de uma coluna ou conjunto de colunas. Podemos usar o método ___describe___ para fazer isso. Isso calculará as colunas __numéricas__ trazendo: média, desvio padrão, valor min e valor max. Você deve usar isso principalmente para visualização no console porque o esquema pode ser mutável.

In [13]:
df.describe("Quantity","UnitPrice","CustomerID").show()

+-------+------------------+------------------+------------------+
|summary|          Quantity|         UnitPrice|        CustomerID|
+-------+------------------+------------------+------------------+
|  count|              3108|              3108|              1968|
|   mean| 8.627413127413128| 4.151946589446603|15661.388719512195|
| stddev|26.371821677029203|15.638659854603892|1854.4496996893627|
|    min|               -24|               0.0|           12431.0|
|    max|               600|            607.49|           18229.0|
+-------+------------------+------------------+------------------+



Há um número de funções estatísticas disponíveis no pacote __StatFunctions__ (acessível usando __stat__ como vemos no bloco de código abaixo). Estes são os métodos DataFrame que você pode usar para calcular uma variedade de coisas diferentes. Por exemplo, você pode calcular quantis exatos ou aproximados de seus dados usando o método ___approxQuantile___:

In [14]:
colName = "UnitPrice"
quantileProbs = [0.5]
relError = 0.05
df.stat.approxQuantile("UnitPrice", quantileProbs, relError)

[2.51]

In [15]:
# Você também pode usar isso para ver uma tabulação cruzada (reduzidos pelo tamanho da saída):
df.stat.crosstab("StockCode", "Quantity").show(2)

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             22578|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 

In [7]:
# Itens frequentes:
df.stat.freqItems(["StockCode", "Quantity"]).show(1) 

+--------------------+--------------------+
| StockCode_freqItems|  Quantity_freqItems|
+--------------------+--------------------+
|[90214E, 20728, 2...|[200, 128, 23, 32...|
+--------------------+--------------------+



A função ___monotonically_increasing_id___ gera um valor único para cada linha, começando com 0:

In [17]:
from pyspark.sql.functions import monotonically_increasing_id
df.select(monotonically_increasing_id()).show(2)

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
+-----------------------------+
only showing top 2 rows



In [18]:
from pyspark.sql.functions import monotonically_increasing_id
df.select(monotonically_increasing_id().alias("Identificador")).show(2)

+-------------+
|Identificador|
+-------------+
|            0|
|            1|
+-------------+
only showing top 2 rows



## Strings

A manipulação de strings é mostrada em quase todos os fluxos de dados, e vale a pena explicar o que você pode fazer com as strings. Você pode estar manipulando arquivos de log executando extração ou substituição de expressões regulares ou verificando a existência de strings simples ou tornando todas as strings maiúsculas ou minúsculas. Vamos começar com a última tarefa porque é a mais simples. A função ___initcap___ irá capitalizar a primeira letra de cada palavra em uma determinada string quando essa palavra é separada de outra por um espaço.

In [8]:
from pyspark.sql.functions import initcap
df.select(initcap(col("Description"))).show(5, False)

# Equivalente em SQL:
# SELECT initcap(Description) FROM dfTable

+-----------------------------------+
|initcap(Description)               |
+-----------------------------------+
|White Hanging Heart T-light Holder |
|White Metal Lantern                |
|Cream Cupid Hearts Coat Hanger     |
|Knitted Union Flag Hot Water Bottle|
|Red Woolly Hottie White Heart.     |
+-----------------------------------+
only showing top 5 rows



In [9]:
# Conversão para maiúsculas e minúsculas
from pyspark.sql.functions import lower, upper
df.select(col("Description"), lower(col("Description")), upper(lower(col("Description")))).show(2, False)

# Equivalente em SQL:
# SELECT Description, lower(Description), Upper(lower(Description)) FROM dfTable

+----------------------------------+----------------------------------+----------------------------------+
|Description                       |lower(Description)                |upper(lower(Description))         |
+----------------------------------+----------------------------------+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|white hanging heart t-light holder|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |white metal lantern               |WHITE METAL LANTERN               |
+----------------------------------+----------------------------------+----------------------------------+
only showing top 2 rows



In [22]:
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim
df.select(
ltrim(lit(" HELLO ")).alias("ltrim"),              # Removendo espaços a esquerda
rtrim(lit(" HELLO ")).alias("rtrim"),              # Removendo espaços a direita   
trim(lit(" HELLO ")).alias("trim"),                # Removendo espaços dos dois lados 
lpad(lit("HELLO"), 3, " ").alias("lp"),            
rpad(lit("HELLO"), 10, " ").alias("rp")).show(2)

# Dado o numero inteiro (n) passado como parâmetro e uma string (s), lpad faz:
# n - len(s) = string mais n espaços.
# Caso o numero (n) seja menor que a string, ele remove caracteres do lado direito da mesma

+------+------+-----+---+----------+
| ltrim| rtrim| trim| lp|        rp|
+------+------+-----+---+----------+
|HELLO | HELLO|HELLO|HEL|HELLO     |
|HELLO | HELLO|HELLO|HEL|HELLO     |
+------+------+-----+---+----------+
only showing top 2 rows



### Expressões regulares

Provavelmente, uma das tarefas executadas com mais frequência é pesquisar a existência de uma string em outra ou substituir todas as menções de uma string por outro valor. Isso geralmente é feito com uma ferramenta chamada ___expressões regulares___ que existe em muitas linguagens de programação. Expressões regulares dão ao usuário a capacidade de especificar um conjunto de regras a serem usadas para extrair valores de uma string ou substituí-los por outros valores.

O Spark aproveita todo o poder das expressões regulares do Java. A sintaxe de expressões regulares do Java se distancia um pouco das outras linguagens de programação, portanto vale a pena revisar antes de colocar qualquer coisa em produção. Existem duas funções principais no Spark que você precisa para executar tarefas de expressão regular: <b>regexp_extract</b> e <b>regexp_replace</b>. Essas funções extraem valores e substituem valores, respectivamente.

In [19]:
from pyspark.sql.functions import regexp_replace

regex_string = "RED|WHITE|BLACK|GREEN|BLUE"
df.select(
regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),
col("Description")).show(3, False)

# Equivalente em SQL:
# SELECT
#     regexp_replace(Description, 'BLACK|WHITE|RED|GREEN|BLUE', 'COLOR') as
#     color_clean, Description
# FROM dfTable

+----------------------------------+----------------------------------+
|color_clean                       |Description                       |
+----------------------------------+----------------------------------+
|COLOR HANGING HEART T-LIGHT HOLDER|WHITE HANGING HEART T-LIGHT HOLDER|
|COLOR METAL LANTERN               |WHITE METAL LANTERN               |
|CREAM CUPID HEARTS COAT HANGER    |CREAM CUPID HEARTS COAT HANGER    |
+----------------------------------+----------------------------------+
only showing top 3 rows



Outra tarefa pode ser substituir determinados caracteres por outros caracteres. Construir isso como uma expressão regular pode ser entediante, portanto, o Spark também fornece a função ___translate___ para substituir esses valores. Isso é feito no nível do caractere e substituirá todas as instâncias de um caractere pelo caractere indexado na cadeia de substituição.

In [20]:
from pyspark.sql.functions import translate
df.select(translate(col("Description"), "LEET", "1337"),col("Description")).show(2, False)

# Equivalente em SQL:
# SELECT translate(Description, 'LEET', '1337'), Description FROM dfTable

+----------------------------------+----------------------------------+
|translate(Description, LEET, 1337)|Description                       |
+----------------------------------+----------------------------------+
|WHI73 HANGING H3AR7 7-1IGH7 HO1D3R|WHITE HANGING HEART T-LIGHT HOLDER|
|WHI73 M37A1 1AN73RN               |WHITE METAL LANTERN               |
+----------------------------------+----------------------------------+
only showing top 2 rows



In [25]:
# Também podemos realizar algo semelhante, como exibir a primeira cor mencionada
from pyspark.sql.functions import regexp_extract
extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(
regexp_extract(col("Description"), extract_str, 1).alias("color_clean"),
col("Description")).show(2)

# Equivalente em SQL:
# SELECT regexp_extract(Description, '(BLACK|WHITE|RED|GREEN|BLUE)', 1),
#     Description
# FROM dfTable

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE|WHITE HANGING HEA...|
|      WHITE| WHITE METAL LANTERN|
+-----------+--------------------+
only showing top 2 rows



In [26]:
# Às vezes, em vez de extrair valores, simplesmente queremos verificar sua existência. Podemos fazer isso com o método 
# contains em cada coluna. Isso retornará um booleano declarando se o valor especificado está na string da coluna:
from pyspark.sql.functions import instr
containsBlack = instr(col("Description"), "BLACK") >= 1
containsWhite = instr(col("Description"), "WHITE") >= 1
df.withColumn("hasSimpleColor", containsBlack | containsWhite)\
    .where("hasSimpleColor")\
    .select("Description").show(3, False)

# Equivalente em SQL:
# SELECT Description FROM dfTable
# WHERE instr(Description, 'BLACK') >= 1 OR instr(Description, 'WHITE') >= 1

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



Vamos trabalhar com isso de maneira mais rigorosa e aproveitar a capacidade do Spark de aceitar um número dinâmico de argumentos. Quando convertemos uma lista de valores em um conjunto de argumentos e os passamos para uma função, usamos um recurso de linguagem chamado ___varargs___. Usando esse recurso, podemos desvendar efetivamente um array de tamanho arbitrário e passá-lo como argumentos para uma função. Isso, juntamente com o select, permite que criemos números arbitrários de colunas dinamicamente.

Com o Python, vamos usar uma função diferente, locate, que retorna o local inteiro. Em seguida, convertemos isso em um booleano antes de usá-lo como o mesmo recurso básico:

In [27]:
from pyspark.sql.functions import expr, locate

simpleColors = ["black", "white", "red", "green", "blue"]

def color_locator(column, color_string):
    return locate(color_string.upper(), column).cast("boolean").alias("is_" + c)

selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns.append(expr("*"))

df.select(*selectedColumns).where(expr("is_white OR is_red")).select("Description").show(3, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



## Datas e Timestamps

Datas e horários são um desafio constante em linguagens de programação e bancos de dados. É sempre necessário acompanhar os fusos horários e garantir que os formatos estejam corretos e válidos. O Spark faz o melhor para manter as coisas simples, concentrando-se explicitamente em dois tipos de informações relacionadas ao tempo. Há datas, que se concentram exclusivamente em datas de calendário e __timestamps__, que incluem informações de data e hora. O Spark, como vimos com nosso conjunto de dados atual, fará o melhor esforço para identificar corretamente os tipos de colunas, incluindo datas e carimbos de data e hora quando ativarmos o inferSchema.

Podemos ver que isso funcionou muito bem com nosso conjunto de dados atual, pois foi capaz de identificar e ler nosso formato de data sem que precisássemos fornecer alguma especificação para isso. Como sugerimos anteriormente, trabalhar com datas e __timestamps__ se relaciona de perto ao trabalho com strings porque geralmente armazenamos nossos timestamps ou datas como strings e as convertemos em tipos de data em tempo de execução. Isso é menos comum quando se trabalha com bancos de dados e dados estruturados, mas é muito mais comum quando estamos trabalhando com arquivos de texto e CSV. 

In [28]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



Embora o Spark faça a leitura de datas ou horas com base no melhor esforço, às vezes não haverá como trabalhar com datas e horários estranhamente formatados. A chave para entender as transformações que você precisará aplicar é garantir que você saiba exatamente que tipo e formato você tem em cada etapa do caminho. Outra pegadinha comum é que a classe ___TimestampType___ do Spark oferece suporte apenas a precisão de segundo nível, o que significa que, se você estiver trabalhando com milissegundos ou microssegundos, precisará solucionar esse problema operando com base neles a longo prazo. 

Qualquer precisão maior ao coagir para um ___TimestampType___ será removida. O Spark pode ser um pouco específico sobre o formato que você tem em um determinado ponto no tempo. É importante ser explícito ao analisar ou converter para garantir que não haja problemas ao fazê-lo. No final do dia, o Spark está trabalhando com datas e timestamps Java e, portanto, está em conformidade com esses padrões.

In [29]:
from pyspark.sql.functions import current_date, current_timestamp

dateDF = spark.range(10)\
    .withColumn("today", current_date())\
    .withColumn("now", current_timestamp())

dateDF.createOrReplaceTempView("dateTable")
dateDF.printSchema()
dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



In [30]:
# Agora que temos um DataFrame simples para trabalhar, vamos adicionar e subtrair cinco dias a partir de hoje. 
# Essas funções selecionam uma coluna e, em seguida, o número de dias para adicionar ou subtrair como os argumentos:

from pyspark.sql.functions import date_add, date_sub
dateDF.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

# Equivalente em SQL:
# SELECT date_sub(today, 5), date_add(today, 5) FROM dateTable

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2019-05-07|        2019-05-17|
+------------------+------------------+
only showing top 1 row



In [31]:
# Outra tarefa comum é observar a diferença entre duas datas. Podemos fazer isso com a função datediff 
# que retornará o número de dias entre duas datas. Na maioria das vezes apenas nos preocupamos com os dias, 
# e como o número de dias varia de mês para mês, também existe uma função, months_between, 
# que fornece o número de meses entre duas datas:

from pyspark.sql.functions import datediff, months_between, to_date

dateDF.withColumn("week_ago", date_sub(col("today"), 7))\
    .select(datediff(col("week_ago"), col("today"))).show(1)

dateDF.select(to_date(lit("2016-01-01")).alias("start"),
              to_date(lit("2017-05-22")).alias("end"))\
            .select(months_between(col("start"), col("end"))).show(1)

# SELECT to_date('2016-01-01'), months_between('2016-01-01', '2017-01-01'),
# datediff('2016-01-01', '2017-01-01')
# FROM dateTable

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row

+--------------------------+
|months_between(start, end)|
+--------------------------+
|              -16.67741935|
+--------------------------+
only showing top 1 row



In [32]:
# A função to_date permite converter uma string em uma data, opcionalmente com um formato especificado. 
# Nós especificamos nosso formato no Java SimpleDateFormat, que será importante para referenciar se você usar esta função:

from pyspark.sql.functions import to_date, lit
spark.range(5).withColumn("date", lit("2017-01-01"))\
    .select(to_date(col("date"))).show(1)

+---------------+
|to_date(`date`)|
+---------------+
|     2017-01-01|
+---------------+
only showing top 1 row



In [33]:
# O Spark não lançará um erro se não puder analisar a data; em vez disso, apenas retornará null. 
# Isso pode ser um pouco complicado em pipelines maiores porque você pode estar esperando seus 
# dados em um formato e obtê-los em outro.

dateDF.select(to_date(lit("2016-20-12")),to_date(lit("2017-12-11"))).show(1)

+---------------------+---------------------+
|to_date('2016-20-12')|to_date('2017-12-11')|
+---------------------+---------------------+
|                 null|           2017-12-11|
+---------------------+---------------------+
only showing top 1 row



Achamos que essa é uma situação especialmente complicada para bugs, pois algumas datas podem corresponder ao formato correto, enquanto outras não. No exemplo anterior, observe como a segunda data aparece como 11 de dezembro em vez do dia correto, 12 de novembro. O Spark não gera um erro porque não pode saber se os dias estão misturados ou se a linha específica está incorreta. Vamos corrigir esse pipeline, passo a passo, e criar uma maneira robusta de evitar totalmente esses problemas. O primeiro passo é lembrar que precisamos especificar nosso formato de data de acordo com o padrão __Java SimpleDateFormat__. Vamos usar duas funções para corrigir isso: ```to_date``` e ```to_timestamp```. 

In [34]:
from pyspark.sql.functions import to_date
dateFormat = "yyyy-dd-MM"

cleanDateDF = spark.range(1).select(\
    to_date(lit("2017-12-11"), dateFormat).alias("date"),\
    to_date(lit("2017-20-12"), dateFormat).alias("date2"))

cleanDateDF.createOrReplaceTempView("dateTable2")

# Equivalente em SQL:
# SELECT to_date(date, 'yyyy-dd-MM'), to_date(date2, 'yyyy-dd-MM'), to_date(date)
# FROM dateTable2

In [35]:
# Agora, vamos usar um exemplo de to_timestamp, que sempre exige que um formato seja especificado:
from pyspark.sql.functions import to_timestamp
cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()

# Equivalente em SQL:
# SELECT to_timestamp(date, 'yyyy-dd-MM'), to_timestamp(date2, 'yyyy-dd-MM')
# FROM dateTable2

+----------------------------------+
|to_timestamp(`date`, 'yyyy-dd-MM')|
+----------------------------------+
|               2017-11-12 00:00:00|
+----------------------------------+



In [ ]:
# A correspondência entre datas e timestamp é simples em todos as linguagens - no SQL, faríamos isso no seguinte maneira:
# SELECT cast(to_date("2017-01-01", "yyyy-dd-MM") as timestamp)

In [36]:
# Depois de termos nossa data ou registro de data e hora no formato e tipo corretos, comparar entre eles 
# é realmente muito fácil. Precisamos apenas usar um tipo de data / timestamp ou especificar nossa string 
# de acordo com o formato correto de aaaa-MM-dd, se estivermos comparando uma data:

cleanDateDF.filter(col("date2") > lit("2017-12-12")).show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [37]:
# Um ponto de menor importância é que também podemos definir isso como uma string, que o Spark analisa para um literal:
cleanDateDF.filter(col("date2") > "'2017-12-12'").show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



Material baseado em exemplos do livro __Spark - The Definitive Guide. Bill Chambers e Matei Zaharia__